In [1]:
import torch
import sys
import time
import numpy as np

### Training model

#### Implementing custom dataset

In [2]:
from dataset import LMDBDataset 

sys.path.append('./config')
from train import train_start_index, train_end_index, data_path, batch_size, val_start_index, val_end_index, is_import_model, model_path, num_epochs, patience_limit

In [3]:
def collate_fn(batch):
    return tuple(zip(*batch))


train_dataset = LMDBDataset(data_path['train'],train_start_index,train_end_index)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True,collate_fn=collate_fn, num_workers=4)

val_dataset = LMDBDataset(data_path['val'],val_start_index,val_end_index)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True,collate_fn=collate_fn, num_workers=4)

#### Setting Model

In [4]:
from model import FasterRCNNResNet50 

In [5]:
model = FasterRCNNResNet50(num_classes=14)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

/home/brandonrafaellovelyno/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [ ]:
if is_import_model:
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    min_val_loss = checkpoint['min_val_loss']
    patience = checkpoint['patience']
else:
    start_epoch = 0
    min_val_loss = np.inf
    patience = 0

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

#### Looping train

In [7]:
for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (images, targets) in enumerate(train_dataloader):
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        running_loss += losses.item()
    
    avg_train_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss}")

    lr_scheduler.step()

    val_running_loss = 0.0
    with torch.no_grad():
        for i, (images, targets) in enumerate(val_dataloader):
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            val_running_loss += losses.item()

    avg_val_loss = val_running_loss / len(val_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Validation Loss: {avg_val_loss}")

    if avg_val_loss < min_val_loss : 
        patience = 0
        min_val_loss = avg_val_loss

        torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': lr_scheduler.state_dict(),
        'patience' : patience,
        'min_val_loss' : min_val_loss
        }, f'checkpoint_epoch_{epoch+1}.pth')

    else :
        patience += 1

    if patience_limit == patience :
        break
    
print("Training complete!")


Epoch [1/20], Average Training Loss: 0.34524448415772707


AttributeError: 'list' object has no attribute 'values'